In [48]:
import os
import re
import time
import pickle
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from IPython.display import clear_output

from dotenv import load_dotenv
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import ClovaXEmbeddings
from langchain_community.chat_models import ChatClovaX
from pymilvus import connections, utility
from langchain_community.vectorstores.milvus import Milvus

In [ ]:
# Milvus 연결 및 환경 설정
load_dotenv(dotenv_path=r"C:\Kill_the_RAG\Project\Aiffel_final_project\.env")

os.environ["NCP_CLOVASTUDIO_API_KEY"] = os.getenv("NCP_CLOVASTUDIO_API_KEY")
os.environ["NCP_CLOVASTUDIO_API_URL"] = os.getenv(
    "NCP_CLOVASTUDIO_API_URL", "https://clovastudio.stream.ntruss.com/"
)

connections.connect(
    alias="default",
    host=os.getenv("MILVUS_HOST", "localhost"),
    port=os.getenv("MILVUS_PORT", "19530"),
)

In [ ]:
# 임베딩 모델 및 LLM 초기화
ncp_embeddings = ClovaXEmbeddings(model="bge-m3")
llm_clova = ChatClovaX(model="HCX-003", max_tokens=2048)  # 맥스 토큰

### 임베딩 pkl 합치기

In [ ]:
# # 임베딩 파일 경로
# folder_path = (
#     r"C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\semifinal_embedding"
# )

# # merge 파일 리스트
# file_names = [f"embedding_batch_{i}.pkl" for i in range(1, 51)]

# merged_embeddings = []
# merged_metadata = []

# # 각 파일을 순회하면서 데이터를 합침
# for file_name in file_names:
#     file_path = os.path.join(folder_path, file_name)
#     print(f"파일 로드 중: {file_path}")
#     with open(file_path, "rb") as f:
#         data = pickle.load(f)
#     # "embeddings"와 "metadata" 키가 존재한다면 리스트에 추가
#     embeddings = data.get("embeddings", [])
#     metadata = data.get("metadata", [])
#     merged_embeddings.extend(embeddings)
#     merged_metadata.extend(metadata)

# # 합쳐진 데이터를 딕셔너리로 구성
# merged_data = {"embeddings": merged_embeddings, "metadata": merged_metadata}

# # 저장할 파일 경로
# output_file = os.path.join(folder_path, "embedding_semifinal.pkl")
# with open(output_file, "wb") as f:
#     pickle.dump(merged_data, f)

# print(f"{output_file}에 merge 완료")

파일 로드 중: C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\semifinal_embedding\embedding_batch_1.pkl
파일 로드 중: C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\semifinal_embedding\embedding_batch_2.pkl
파일 로드 중: C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\semifinal_embedding\embedding_batch_3.pkl
파일 로드 중: C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\semifinal_embedding\embedding_batch_4.pkl
파일 로드 중: C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\semifinal_embedding\embedding_batch_5.pkl
파일 로드 중: C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\semifinal_embedding\embedding_batch_6.pkl
파일 로드 중: C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\semifinal_embedding\embedding_batch_7.pkl
파일 로드 중: C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\semifinal_embedding\embedding_batch_8.pkl
파일 로드 중: C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\semifinal_embedding\embedding_batch_9.pkl
파일 로드 중: C:\Kill_the_RAG\Project\Aiffel_final_

In [ ]:
# # 데이터 로드 및 전처리
# df = pd.read_csv(
#     r"C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\aiffel_book_250318_semifinal(filled_contents).csv"
# )

In [ ]:
# pkl 임베딩 파일에서 데이터 불러오기 (원본 텍스트 및 메타데이터)
embedding_file = r"C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\semifinal_embedding\embedding_semifinal.pkl"
if os.path.exists(embedding_file):
    with open(embedding_file, "rb") as f:
        saved_data = pickle.load(f)
    all_text_embedding_pairs = saved_data["embeddings"]
    all_metadata_list = saved_data["metadata"]
    print("임베딩 데이터 불러오기")
else:
    raise FileNotFoundError(f"임베딩 파일을 찾을 수 없습니다: {embedding_file}")

# Milvus issue - 한글 메타데이터 키를 영어로 매핑 (임베딩된 vector_doc_columns도 포함)
metadata_mapping = {
    "ISBN": "ISBN",
    "페이지": "page",
    "가격": "price",
    "제목": "title",
    "저자": "author",
    "분류": "category",
    "저자소개": "author_intro",
    "책소개": "book_intro",
    "목차": "table_of_contents",
    "출판사리뷰": "publisher_review",
    "추천사": "recommendation",
}

# pkl 메타데이터에 적용: 모든 메타데이터를 영어 키로 재구성
all_metadata_list_mapped = []
for meta in all_metadata_list:
    mapped_meta = {metadata_mapping.get(key, key): value for key, value in meta.items()}
    all_metadata_list_mapped.append(mapped_meta)

# documents 리스트 구성: 원본 텍스트와 영어 메타데이터 사용
documents = [
    Document(page_content=pair[0], metadata=meta)
    for pair, meta in zip(all_text_embedding_pairs, all_metadata_list_mapped)
]

임베딩 데이터 불러오기


In [ ]:
# Milvus 벡터 DB 구축
collection_name = "book_rag_db"
if utility.has_collection(collection_name):
    utility.drop_collection(collection_name)

vectorstore = Milvus(
    embedding_function=ncp_embeddings,
    collection_name=collection_name,
    connection_args={"host": "localhost", "port": "19530"},
    auto_id=True,
)

texts = [pair[0] for pair in all_text_embedding_pairs]
embeds = [pair[1] for pair in all_text_embedding_pairs]


def precomputed_embed_documents(cls, input_texts):
    if input_texts != texts:
        raise ValueError(
            "ERROR : 입력 텍스트 순서가 사전 계산된 임베딩과 일치하지 않음"
        )
    return embeds


ClovaXEmbeddings.embed_documents = classmethod(precomputed_embed_documents)

vectorstore.add_texts(
    texts=texts, metadatas=all_metadata_list_mapped, embeddings=embeds
)

[456769738505236100,
 456769738505236101,
 456769738505236102,
 456769738505236103,
 456769738505236104,
 456769738505236105,
 456769738505236106,
 456769738505236107,
 456769738505236108,
 456769738505236109,
 456769738505236110,
 456769738505236111,
 456769738505236112,
 456769738505236113,
 456769738505236114,
 456769738505236115,
 456769738505236116,
 456769738505236117,
 456769738505236118,
 456769738505236119,
 456769738505236120,
 456769738505236121,
 456769738505236122,
 456769738505236123,
 456769738505236124,
 456769738505236125,
 456769738505236126,
 456769738505236127,
 456769738505236128,
 456769738505236129,
 456769738505236130,
 456769738505236131,
 456769738505236132,
 456769738505236133,
 456769738505236134,
 456769738505236135,
 456769738505236136,
 456769738505236137,
 456769738505236138,
 456769738505236139,
 456769738505236140,
 456769738505236141,
 456769738505236142,
 456769738505236143,
 456769738505236144,
 456769738505236145,
 456769738505236146,
 456769738505

In [ ]:
# Dense Retriever 및 RetrievalQA 체인 구축 - 추후 sparse 등 추가하여 페르소나 별 다르게 지정 가능
dense_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
dpr_qa_chain = RetrievalQA.from_chain_type(
    llm=llm_clova, retriever=dense_retriever, return_source_documents=True
)


def extract_field(text, field_name):
    # "필드명 : 값" 형태에서 값을 추출 : 청크에 들어가있는 원본데이터의 형태 중 특정 값만 추출하기 위해
    pattern = rf"{re.escape(field_name)}\s*:\s*(.*)"
    match = re.search(pattern, text)
    return match.group(1).strip() if match else ""


MIN_INFO_LENGTH = 10

In [ ]:
# 공용 시스템 프롬프트 템플릿
common_multi_turn_prompt = """
[대화 맥락]
사용자 대화 내역:
{{history}}
사용자의 최신 질문: "{{query}}"

[역할 및 목표]
{role_instructions}

[MultiChat 출력 형식 - 반드시 아래 내용만 출력]
검색 확률: {{{{숫자, 0~1 사이 (예: 0.75)}}}}
기본 검색 쿼리: "{{{{검색 쿼리, 예: 흥미로운 조선시대 역사적 사건을 다룬 한국사 서적}}}}"
추가 질문: "{{{{추가 질문, 정보 보완 필요 시 구체적으로; 충분하면 빈 문자열}}}}"

[출력 예시]
검색 확률: 0.75
기본 검색 쿼리: "이해하기 쉬운 쉬운 역사책"
추가 질문: "혹시 어느 시대 역사책을 찾고 있나요?"
"""

common_final_query_prompt = """
[전체 대화 요약]
{{history}}

[이전 기본 검색 쿼리]
{{fallback}}

[이전 검색 확률]
{{score}}

[역할 및 목표]
{role_instructions}

[최종 출력 형식 - 반드시 아래 내용만 출력]
검색 확률: {{score}}
검색 쿼리: "{{{{최종 검색 쿼리}}}}"
책 제목: "{{{{책 제목}}}}"
저자: "{{{{저자}}}}"
추천 이유: "{{{{추천 이유, 정보 부족 시 '추천 이유 정보 없음'}}}}"

[출력 예시]
검색 확률: "0.75"
검색 쿼리: "흥미로운 조선시대 역사적 사건과 인물을 다룬 한국사 서적"
책 제목: "조선 왕조 실록"
저자: "홍길동"
추천 이유: "조선시대의 정치, 문화, 경제적 요소를 균형 있게 반영하여 심층적 역사 이해에 도움을 줄 수 있음"
"""

In [ ]:
# 페르소나별 역할 지정 시스템 프롬프트
literature_role = (
    "너는 감성적이고 문학적인 도서 추천 챗봇이다. 대화를 통해 사용자의 선호하는 장르, 작가, 시대를 파악하고, "
    "감성적인 표현을 활용하여 책을 추천해라."
)

science_role = (
    "너는 정확하고 논리적인 과학/기술 도서 추천 챗봇이다. 대화를 통해 사용자의 관심 분야(컴퓨터, 생물, 물리 등)와 "
    "난이도를 파악하여, 명확하고 구체적인 정보를 바탕으로 책을 추천해라."
)

In [ ]:
# RAG 파이프라인 클래스 정의
class BaseRAGPipeline:
    def __init__(self, config, llm, retriever, qa_chain, documents):
        self.config = config
        self.llm = llm
        self.retriever = retriever
        self.qa_chain = qa_chain
        self.documents = documents
        self.query_history = []
        self.user_preferences = defaultdict(list)

        self.multi_turn_chain = LLMChain(
            llm=self.llm,
            prompt=PromptTemplate(
                input_variables=["history", "query"],
                template=common_multi_turn_prompt.format(
                    role_instructions=config["role_instructions"]
                ),
            ),
        )
        self.final_query_chain = LLMChain(
            llm=self.llm,
            prompt=PromptTemplate(
                input_variables=["history", "fallback", "score"],
                template=common_final_query_prompt.format(
                    role_instructions=config["role_instructions"]
                ),
            ),
        )

    def robust_parse_llm_response(self, response_text):
        score_match = re.search(r"검색\s*확률\s*[:：]\s*([\d\.]+)", response_text)
        search_score = float(score_match.group(1)) if score_match else None

        query_match = re.search(
            r"기본\s*검색\s*쿼리\s*[:：]\s*\"([^\"]+)\"", response_text
        )
        search_query = query_match.group(1).strip() if query_match else None

        follow_match = re.search(r"추가\s*질문\s*[:：]\s*\"([^\"]*)\"", response_text)
        follow_up_question = follow_match.group(1).strip() if follow_match else ""

        return search_score, search_query, follow_up_question

    def generate_answer(self, query):
        result = self.qa_chain.invoke(query)
        source_docs = result["source_documents"]

        # ISBN 기준으로 동일 도서 문서들을 그룹화 (pkl 임베딩 시 metadata에 포함시킨 영문키를 활용해서 뽑음) - df 원본 text로 안 뽑음
        retrieved_isbns = set()
        for doc in source_docs:
            isbn = doc.metadata.get("ISBN")
            if isbn:
                retrieved_isbns.add(isbn)

        aggregated_docs = []
        for isbn in retrieved_isbns:
            book_docs = [
                doc for doc in self.documents if doc.metadata.get("ISBN") == isbn
            ]
            if not book_docs:
                continue
            aggregated_text = "\n".join([doc.page_content for doc in book_docs])
            aggregated_docs.append(
                Document(page_content=aggregated_text, metadata=book_docs[0].metadata)
            )

        formatted_answers = []
        for doc in aggregated_docs:
            metadata = doc.metadata
            title = metadata.get("title") or extract_field(doc.page_content, "제목")
            author = metadata.get("author") or extract_field(doc.page_content, "저자")
            extra_field = ""
            if self.config.get("persona") == "Science":
                extra_field = metadata.get("year") or extract_field(
                    doc.page_content, "출판년도"
                )

            # aggregated_text에서 '책소개', '출판사리뷰', '추천사' 추출 (영어 키로 변환되지 않은 본문은 df 시절의 포맷 그대로)
            # 나중에 추천 이유 중 어디에 집중할지 extract_field을 통해 구현 가능
            aggregated_text = doc.page_content
            book_intro = extract_field(aggregated_text, "책소개")
            publisher_review = extract_field(aggregated_text, "출판사리뷰")
            recommendation_field = extract_field(aggregated_text, "추천사")

            # 우선 '추천사'가 충분하면 사용하고, 없으면 '출판사리뷰'와 '책소개'를 조합 - 추천사가 제일 없는데 쿼리티는 가장 좋기 때문 : 시간을 단축시키고 싶으면 책소개를 우선으로
            if (
                recommendation_field
                and len(recommendation_field.strip()) >= MIN_INFO_LENGTH
            ):
                combined_info = recommendation_field
            elif publisher_review or book_intro:
                combined_info = (publisher_review or "") + "\n" + (book_intro or "")
            else:
                combined_info = ""

            if not combined_info or len(combined_info.strip()) < MIN_INFO_LENGTH:
                reason = "추천 이유 정보 없음"
            else:
                reason_prompt = (
                    f"다음 정보를 참고하여, 이 책이 추천되는 이유를 간결하고 명확하게 요약해라. "
                    f"책의 주요 특징과 강점을 중심으로 설명해라.\n\n정보:\n{combined_info}"
                )
                reason_response = self.llm.invoke(reason_prompt)
                generated_reason = reason_response.text().strip()
                if (
                    not generated_reason
                    or len(generated_reason) < 10
                    or "추천 이유 정보 없음" in generated_reason
                ):
                    reason = "추천 이유 정보 없음"
                else:
                    reason = generated_reason

            if self.config.get("persona") == "Science":
                formatted = f"{title}\n{author}\n출판년도: {extra_field if extra_field else '정보 없음'}\n추천 이유: {reason}"
            else:
                formatted = f"{title}\n{author}\n추천 이유: {reason}"
            formatted_answers.append(formatted)

        answer = "\n\n".join(formatted_answers)
        return answer, None

    def search_and_generate_answer(self, query):
        while True:
            query_summary = "\n".join(
                self.query_history[-5:]
            )  # 이전 5개의 멀티턴 대화만 요약에 사용
            prompt_vars = {"history": query_summary, "query": query}
            search_decision_dict = self.multi_turn_chain.invoke(prompt_vars)
            response_text = search_decision_dict["text"].strip()
            print("\n[🔍 LLM 응답 확인]\n", response_text)

            search_score, base_search_query, follow_up_question = (
                self.robust_parse_llm_response(response_text)
            )
            print(
                f"\n[디버그] 파싱 결과: 검색 확률={search_score}, 기본 검색 쿼리='{base_search_query}', 추가 질문='{follow_up_question}'"
            )

            if search_score is None:
                print("\n[LLM 응답 파싱 실패: 추가 정보 필요]")
                extra_info = input("추가 정보를 입력해주세요: ")
                self.query_history.append(f"사용자(추가): {extra_info}")
                query = f"{query} {extra_info}"
                continue

            if search_score >= 0.8 and base_search_query:
                final_search_query = self.final_query_chain.invoke(
                    {
                        "history": "\n".join(self.query_history),
                        "fallback": base_search_query,
                        "score": search_score,
                    }
                )["text"].strip()
                print(f"\n[최종 검색 쿼리 생성]: {final_search_query}")

                answer, sources = self.generate_answer(final_search_query)
                if sources:
                    book_info = "\n".join([f"- {title}" for title in sources])
                    answer_with_info = f"{answer}\n\n[책 정보]\n{book_info}"
                    print("\n[책 정보]\n", book_info)
                else:
                    answer_with_info = answer
                return answer_with_info

            if follow_up_question:
                print(f"\n[보충 질문: {follow_up_question}]")
                self.query_history.append(f"AI: {follow_up_question}")
                user_response = input("\n사용자 응답: ")
                self.query_history.append(f"사용자: {user_response}")
                if (
                    "장르" in follow_up_question
                    or "작가" in follow_up_question
                    or "시대" in follow_up_question
                ):
                    self.user_preferences["literature"].append(user_response)
                else:
                    self.user_preferences["misc"].append(user_response)
                print("\n[사용자 선호도 업데이트 완료!]")
                query = f"{query} {follow_up_question} {user_response}"
                continue

            if search_score < 0.8 or not base_search_query:
                print("\n[검색 확률 낮거나 검색 쿼리 없음: 추가 정보 필요]")
                extra_info = input("추가 정보를 입력해주세요: ")
                self.query_history.append(f"사용자(추가): {extra_info}")
                query = f"{query} {extra_info}"
                continue

    def interactive_multi_turn_qa(self):
        while True:
            clear_output(wait=True)
            print(
                f"멀티턴 AI 기반 책 추천 시스템 - {self.config.get('persona', 'Default')} 페르소나 (종료하려면 'quit' 입력)"
            )
            print("-" * 50)
            query = input("질문을 입력하세요: ")
            if query.lower() == "quit":
                print("\n[대화 저장 중...]")
                print("대화 저장 완료")
                import sys

                sys.exit()
            self.query_history.append(f"사용자: {query}")
            answer = self.search_and_generate_answer(query)
            print("\n[AI의 답변]")
            print(answer)
            self.query_history.append(f"AI: {answer}")
            input("\n-> 계속하려면 Enter를 누르세요...")

In [ ]:
# 페르소나별 RAG 파이프라인 클래스 정의 - 추후 추가 가능


class LiteratureRAGPipeline(BaseRAGPipeline):
    def __init__(self, llm, retriever, qa_chain, documents):
        config = {"persona": "Literature", "role_instructions": literature_role}
        super().__init__(config, llm, retriever, qa_chain, documents)


class ScienceRAGPipeline(BaseRAGPipeline):
    def __init__(self, llm, retriever, qa_chain, documents):
        config = {"persona": "Science", "role_instructions": science_role}
        super().__init__(config, llm, retriever, qa_chain, documents)

In [ ]:
# 실행 함수
# 탬플릿에서 선택지를 더 늘리기 + 설명 덧붙이기
def main():
    print("페르소나 선택:")
    print("1. 예술/문학")
    print("2. 과학/기술")
    choice = input("원하는 페르소나 번호를 입력하세요 (1 또는 2): ").strip()

    if choice == "1":
        pipeline = LiteratureRAGPipeline(
            llm_clova, dense_retriever, dpr_qa_chain, documents
        )
    elif choice == "2":
        pipeline = ScienceRAGPipeline(
            llm_clova, dense_retriever, dpr_qa_chain, documents
        )
    else:
        print("잘못된 선택입니다. 기본 예술/문학 페르소나로 실행합니다.")
        pipeline = LiteratureRAGPipeline(
            llm_clova, dense_retriever, dpr_qa_chain, documents
        )

    pipeline.interactive_multi_turn_qa()


if __name__ == "__main__":
    main()

멀티턴 AI 기반 책 추천 시스템 - Science 페르소나 (종료하려면 'quit' 입력)
--------------------------------------------------

[🔍 LLM 응답 확인]
 검색 확률 : 0.85
기본 검색 쿼리 : "AI 에이전트 관련도서"
추가 질문 : "난이도가 있는 책을 원하시나요? 아니면 입문서를 원하시나요?"

[디버그] 파싱 결과: 검색 확률=0.85, 기본 검색 쿼리='AI 에이전트 관련도서', 추가 질문='난이도가 있는 책을 원하시나요? 아니면 입문서를 원하시나요?'

[최종 검색 쿼리 생성]: 검색 확률 : 0.85
검색 쿼리 : "AI 에이전트 관련 도서"
책 제목 : "인공지능 시대의 비즈니스 전략" 
저자 : 정도희 , 조민호
추천 이유 : 인공지능 기술이 기업의 경영 환경과 일하는 방식을 어떻게 변화시키는지 다양한 사례를 통해 보여주며, 새로운 비즈니스 기회를 탐색할 수 있는 인사이트를 제공함으로써 AI 에이전트 개발에도 큰 도움이 될 것임 

책 제목 : "머신러닝 교과서 with 파이썬, 사이킷런, 텐서플로"
저자 : 세바스티안 라시몽
추천 이유 : 머신러닝 기초부터 심화까지 포괄적으로 다루고 있으며, 파이썬, 사이킷런, 텐서플로 라이브러리를 활용한 예제 코드를 수록하고 있어 실제 AI 에이전트 개발에 유용하게 활용할 수 있음

[AI의 답변]
파이썬으로 배우는 머신러닝
김화종 지음
출판년도: 정보 없음
추천 이유: 이 책은 기업 현장에서 디지털 전환, 인공지능 도입, 데이터 분석을 담당하는 실무자를 위한 학습서로, 파이썬 프로그래밍과 머신러닝을 빠르게 배울 수 있는 입문서입니다. 

파이썬의 기초부터 시작하여 머신러닝의 주요 개념을 명확하게 이해할 수 있도록 구성되어 있으며, 실전에서 사용할 수 있는 핵심 예제들을 소개하여 현업에서 바로 활용할 수 있도록 하였습니다. 

따라서, 이 책은 파이썬과 머신러닝을 배우고자 하는 초보자나 입문자에게 추천되며, 기업 현장에서 디지털 전환, 인공지능 도

KeyboardInterrupt: Interrupted by user

In [ ]:
result = dpr_qa_chain.invoke({"query": "..."})
print(len(result["source_documents"]))

3


In [26]:
# pd.set_option('display.max_rows', None)      # 모든 행 보기
# pd.set_option('display.max_columns', None)   # 모든 열 보기
pd.set_option("display.max_colwidth", None)  # 열 내용 길이 제한 없음

In [49]:
# df.loc[
#     df["제목"].str.contains("죄와 벌", na=False),
#     ["제목", "책소개", "출판사리뷰", "저자", "저자소개", "분류"],
# ]

In [20]:
pd.reset_option("all")

C:\Users\computer\AppData\Local\Temp\ipykernel_23808\3365680546.py:1: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option("all")
C:\Users\computer\AppData\Local\Temp\ipykernel_23808\3365680546.py:1: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option("all")
